In [ ]:
%pip -q install git+https://github.com/huggingface/transformers
%pip install -q datasets loralib sentencepiece bitsandbytes accelerate xformers einops

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from google.colab import userdata

userdata.get('Polyjuiceai')

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2",
                                          use_auth_token=True,)

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                            #  load_in_4bit=True
                                             load_in_8bit=True
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:769: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
# To create a text generation pipeline

# set_up 1: temperature: 0.8, top_p: 0.9, do_sample= True
# set_up 2: temperature: 0.5, top_p: 0.7, do_sample= False
# set_up 3: temperature: 0.1, top_p: 0.6, do_sample= False

pipe = pipeline("text-generation", # specify the task for the pipeline
                model = model,
                tokenizer = tokenizer,
                torch_dtype = torch.bfloat16, # data type for PyTorch tensors
                max_length=1024,
                temperature=0.1,
                top_p=0.6,
                repetition_penalty=1.15,
                max_new_tokens=512,
                device_map = 'auto',
                do_sample = False,
                top_k = 50,
                eos_token_id = tokenizer.eos_token_id,
                pad_token_id = tokenizer.eos_token_id,
               )

In [ ]:
import textwrap
# to format the response
# textwrap: Used to wrap or fill text into a specified width. This is helpful for formatting output text to make it more readable

def wrap_text(text, width=150):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')
    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [ ]:
B_TOKEN, E_TOKEN = "<s>", "</s>"
INSTRUCT = "### Instruction:\n"
QUESTION = "\n\n### question:\n"
RESPONSE = "\n\n### Response:\n"

DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature."""


# Creates a complete prompt
def create_prompt(user_query, system_prompt):
  prompt_template = B_TOKEN + INSTRUCT + system_prompt + QUESTION + user_query + RESPONSE + E_TOKEN
  return prompt_template


def generate_response(query, system_prompt=DEFAULT_SYSTEM_PROMPT):
    prompt = create_prompt(query, system_prompt)
    response = pipe(prompt)
    final_response = response[0]["generated_text"][len(prompt):]

    return final_response


In [ ]:
%%time

revised_system_prompt = """You are a knowledgeable, helpful, respectful, and honest assistant specializing in medical expertise and artificial intelligence. Always provide thorough and accurate information, ensuring your answers are as detailed and helpful as possible while maintaining safety and ethical standards. Your answers should avoid harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Ensure that your responses are socially unbiased, positive in nature, and supported by relevant knowledge or research. When appropriate, provide examples, explanations, and context to enhance understanding.\
The generated responses must be accurate, precise, coherent, and consistent with the questions. They should be brief and not too long. For more advanced questions, provide a very short background and a concise summary while giving enough details. The next criterion in double asterisks is very important to follow: **Do not exceed the maximum length of the response. The maximum length of the generated response is defined as 512 new tokens; try not to exceed this and give a complete answer within this limit. Ensure that sentences are complete and ideas are fully conveyed within this constraint. If the response is too long, ensure to generate responses briefly and summarize the key points effectively while maintaining clarity and completeness. Always prioritize brevity without sacrificing essential details.**"""

query = "What are large language models?"
res = generate_response(query)
print(f"\n {wrap_text(res)}\n")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (


  Large language models are artificial intelligence (AI) systems designed to understand and generate human-like text based on the input they receive.
They're trained on vast amounts of data, allowing them to learn patterns and relationships within language. These models can be used for various
applications such as text generation, translation, summarization, and even answering questions like this one! They don't have the ability to
physically exist or perform tasks outside of generating text, but they can certainly help us with understanding and creating written language.

CPU times: user 30.6 s, sys: 158 ms, total: 30.8 s
Wall time: 31.4 s


### Create HR Related General QA Dictionary

In [ ]:
file_path = str(input("Enter the file path: "))

Enter the file path: /content/HR_QA.csv


In [ ]:
import pandas as pd

csv_file_path = file_path
df = pd.read_csv(csv_file_path)
df = df.dropna()

print("These are the questions: \n")
for ind in range(len(df)):
    print(f"Q {ind+1}: {df.loc[ind, 'Question']}")

These are the questions: 

Q 1: Can you explain the difference between supervised and unsupervised learning in the context of AI?
Q 2: What are some common applications of AI in the medical field today, and how do they improve patient care?
Q 3: How do you ensure the quality and accuracy of medical data used for training AI models?
Q 4: What are the key ethical considerations when implementing AI in healthcare, and how would you address them?
Q 5: Can you describe the concept of a neural network and its role in AI?
Q 6: Can you name the major organs in the human body and their primary functions?
Q 7: What is the difference between acute and chronic conditions?
Q 8: What are the normal ranges for vital signs such as blood pressure, heart rate, and respiratory rate?
Q 9: How would you explain the difference between bacterial and viral infections to a patient?
Q 10: What are the steps involved in conducting a standard physical examination?
Q 11: Can you explain the role of the amygdala in

In [ ]:
def create_question_dict(questions_file_path):
  qfile_path = questions_file_path
  dfQ = pd.read_csv(qfile_path)
  qlist = [dfQ.loc[i, 'Question'] for i in range(len(dfQ)) ]
  qa_dict = {key: None for key in qlist}

  return qa_dict


In [ ]:
revised_system_prompt = """You are a knowledgeable, helpful, respectful, and honest assistant specializing in medical expertise and artificial intelligence. Always provide thorough and accurate information, ensuring your answers are as detailed and helpful as possible while maintaining safety and ethical standards. Your answers should avoid harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Ensure that your responses are socially unbiased, positive in nature, and supported by relevant knowledge or research. When appropriate, provide examples, explanations, and context to enhance understanding.\
The generated responses must be accurate, precise, coherent, and consistent with the questions. They should be brief and not too long. For more advanced questions, provide a very short background and a concise summary while giving enough details. The next criterion in double asterisks is very important to follow: **Do not exceed the maximum length of the response. The maximum length of the generated response is defined as 512 new tokens; try not to exceed this and give a complete answer within this limit. Ensure that sentences are complete and ideas are fully conveyed within this constraint. If the response is too long, ensure to generate responses briefly and summarize the key points effectively while maintaining clarity and completeness. Always prioritize brevity without sacrificing essential details.**"""

questions_file_path = file_path
Q_dictionary = create_question_dict(questions_file_path)

for k in Q_dictionary.keys():
  query = str(k)
  res = generate_response(query, revised_system_prompt)
  final_res = wrap_text(res)
  Q_dictionary.update({k : final_res})


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentat

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: Can you explain the difference between supervised and unsupervised learning in the context of AI?
 response: **Supervised learning** refers to a type of machine learning where an algorithm learns from labeled data. In other words, it's trained on input-output
pairs, meaning each example comes with its correct answer. This approach allows the model to understand patterns and relationships in the data,
enabling it to make predictions based on new, unseen data. Examples include regression analysis for continuous values and classification algorithms
like logistic regression and support vector machines (SVM) for categorical variables.

On the other hand, **unsupervised learning** does not rely on labeled data. Instead, it looks for inherent structures and hidden patterns within the
data itself. Unsupervised methods aim to discover underlying relationships among the inputs, often used for clustering similar items together or
dimensionality reduction techniques such as Principal Component

In [ ]:
json_file_name = 'QA_HRGeneral.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'QA_HRGeneral.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

#Create HR Advanced Paper Related QA Dictionary

In [ ]:
file_path = str(input("Enter the file path: "))

Enter the file path: /content/Advanced_Medical_QA.csv


In [ ]:
csv_file_path = file_path
df = pd.read_csv(csv_file_path)
df = df.dropna()

print("These are the questions: \n")
for ind in range(len(df)):
    print(f"Q {ind+1}: {df.loc[ind, 'Question']}")

These are the questions: 

Q 1: How do enhancer-promoter interactions mediated by CTCF and cohesin contribute to transcriptional regulation, and what are the implications of CTCF depletion on TAD structure and gene expression as described in the paper?
Q 2: What are the distinct advantages and limitations of the various 3C-based and imaging-based techniques (such as Hi-C, SPRITE, and super-resolution microscopy) discussed in the paper for studying 3D genome architecture and enhancer-promoter interactions?
Q 3: How do the recent advancements in single-cell ATAC-seq and high-throughput sequencing-based reporter assays contribute to our understanding of cell type-specific cis-regulatory elements and their functional roles in gene transcription?
Q 4: What insights have been gained from using CRISPR-based epigenome-editing technologies in validating the activity of enhancers, and how do these findings impact our understanding of enhancer dynamics and gene regulation in different cellular co

In [ ]:
def create_question_dict(questions_file_path):
  qfile_path = questions_file_path
  dfQ = pd.read_csv(qfile_path)
  qlist = [dfQ.loc[i, 'Question'] for i in range(len(dfQ)) ]
  qa_dict = {key: None for key in qlist}

  return qa_dict


In [ ]:
revised_system_prompt = """You are a knowledgeable, helpful, respectful, and honest assistant specializing in medical expertise and artificial intelligence. Always provide thorough and accurate information, ensuring your answers are as detailed and helpful as possible while maintaining safety and ethical standards. Your answers should avoid harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Ensure that your responses are socially unbiased, positive in nature, and supported by relevant knowledge or research. When appropriate, provide examples, explanations, and context to enhance understanding.\
The generated responses must be accurate, precise, coherent, and consistent with the questions. They should be brief and not too long. For more advanced questions, provide a very short background and a concise summary while giving enough details. The next criterion in double asterisks is very important to follow: **Do not exceed the maximum length of the response. The maximum length of the generated response is defined as 512 new tokens; try not to exceed this and give a complete answer within this limit. Ensure that sentences are complete and ideas are fully conveyed within this constraint. If the response is too long, ensure to generate responses briefly and summarize the key points effectively while maintaining clarity and completeness. Always prioritize brevity without sacrificing essential details.**"""

questions_file_path = file_path
Q_dictionary = create_question_dict(questions_file_path)

for k in Q_dictionary.keys():
  query = str(k)
  res = generate_response(query, revised_system_prompt)
  final_res = wrap_text(res)
  Q_dictionary.update({k : final_res})


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentat

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: How do enhancer-promoter interactions mediated by CTCF and cohesin contribute to transcriptional regulation, and what are the implications of CTCF depletion on TAD structure and gene expression as described in the paper?
 response: *ctcf and cohesin play crucial roles in shaping chromatin architecture through enhancer-promoter interactions.*
**CTCF (CCCTC-binding factor)** is a versatile transcription factor that binds to specific DNA sequences, known as insulator elements, and functions
as a boundary element to prevent the spread of heterochromatin and enhance compartmentalization of chromatin domains. It also interacts with the
cohesin complex, which is involved in chromosome condensation during mitosis and meiosis, but in interphase, it helps form topologically associating
domains (TADs) by facilitating loop extrusion.

**Enhancer-promoter interactions** are essential for proper gene expression. Enhancers are cis-acting regulatory elements located far from the
promoters they reg

In [ ]:
json_file_name = 'QA_AdvancedMedicalAI.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'QA_dvancedMedicalAI.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)